## 试题说明

### 任务描述
近年来，随着人工智能的发展，其在语音识别、自然语言处理、图像与视频分析等诸多领域取得了巨大成功。随着政府对环境保护的呼吁，垃圾分类成为一个亟待解决的问题，本次竞赛将聚焦在垃圾图片的分类，利用人工智能技术，对居民生活垃圾图片进行检测，找出图片中有哪些类别的垃圾。
要求参赛者给出一个算法或模型，对于给定的图片，检测出图片中的垃圾类别。给定图片数据，选手据此训练模型，为每张测试数据预测出最正确的类别。

### 数据说明
本竞赛所用训练和测试图片均来自生活场景。总共四十个类别，类别和标签对应关系在训练集中的dict文件里。图片中垃圾的类别，格式是“一级类别/二级类别”，二级类别是具体的垃圾物体类别，也就是训练数据中标注的类别，比如一次性快餐盒、果皮果肉、旧衣服等。一级类别有四种类别：可回收物、厨余垃圾、有害垃圾和其他垃圾。

数据文件包括训练集(有标注)和测试集(无标注)，训练集的所有图片分别保存在train文件夹下面的0-39个文件夹中，文件名即类别标签，测试集共有400张待分类的垃圾图片在test文件夹下，testpath.txt保存了所有测试集文件的名称，格式为：name+\n。

![](https://ai-studio-static-online.cdn.bcebos.com/cec625e7b61d459fa13ed2b822817bfd57030ad23c524f1ab93b3e7108d75d78)
![](https://ai-studio-static-online.cdn.bcebos.com/df06e2956f2044fab21fde2a9602abfdb9d4720861d54c66b2e079f3e64000a7)

### 提交答案
考试提交，需要提交**模型代码项目版本**和**结果文件**。结果文件为TXT文件格式，命名为model_result.txt，文件内的字段需要按照指定格式写入。

提交结果的格式如下：
1. 每个类别的行数和测试集原始数据行数应一一对应，不可乱序。
2. 输出结果应检查是否为400行数据，否则成绩无效。
3. 输出结果文件命名为model_result.txt，一行一个类别标签（数字）


样例如下：

···

35

3

2

37

10

3

26

4

34

21

···


In [1]:
### 开始答题

In [2]:
!pip install paddlehub==1.8.1 -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [3]:
!unzip -q -o ./data/data35095/train.zip -d ./data/rubbish

In [4]:
import os
import zipfile
import random
import json
import cv2
import numpy as np
from PIL import Image
from PIL import ImageEnhance
import paddle
import paddle.fluid as fluid
from paddle.fluid.dygraph import Linear
from paddle.fluid.dygraph import Pool2D, Conv2D
import matplotlib.pyplot as plt

In [5]:
'''
参数配置
'''
train_parameters = {
    "src_path":"/home/aistudio/data/data35095/sisters.zip",       #原始数据集路径
    "target_path":"/home/aistudio/data/rubbish/",                  #要解压的路径
    "label_list_path": "/home/aistudio/data/rubbish/train/label.txt",   #label.txt路径
    "train_list_path": "/home/aistudio/data/rubbish/train/train.txt",   #train.txt路径
    "eval_list_path": "/home/aistudio/data/rubbish/train/eval.txt",     #eval.txt路径
    "readme_path":"/home/aistudio/data/rubbish/readme.json",
    "label_dict":{},
}

In [6]:
def get_data_list(target_path,train_list_path,eval_list_path):
    '''
    生成数据列表
    '''
    
    #存放所有类别的信息
    class_detail = []
    #类别信息字典
    class_path = "data/rubbish/garbage_dict.json"
    f = open(class_path, 'r')
    dict_data = json.load(f)
    #获取所有类别保存的文件夹名称
    data_list_path=target_path+"train/"
    class_dirs = os.listdir(data_list_path)  
    #总的图像数量
    all_class_images = 0
    #存放类别标签
    class_label=0
    #存放类别数目
    class_dim = 0
    #存储要写进eval.txt和train.txt中的内容
    label_list=[]
    trainer_list=[]
    eval_list=[]
    #读取每个类别，0-39类
    for class_dir in class_dirs:
        if class_dir != ".DS_Store":
            class_dim += 1
            class_label = int(class_dir)
            #每个类别的信息
            class_detail_list = {}
            eval_sum = 0
            trainer_sum = 0
            #统计每个类别有多少张图片
            class_sum = 0
            #获取类别路径 
            path = data_list_path  + class_dir
            # 获取所有图片
            img_paths = os.listdir(path)
            for img_path in img_paths:                                  # 遍历文件夹下的每个图片
                name_path = class_dir + '/' + img_path                       # 每张图片的路径
                if class_sum % 200 == 0:                                # 每100张图片取一个做验证数据
                    eval_sum += 1                                       # eval_sum为测试数据的数目
                    eval_list.append(name_path + " %d" % class_label + "\n")
                else:
                    trainer_sum += 1 
                    trainer_list.append(name_path + " %d" % class_label + "\n")#trainer_sum测试数据的数目
                class_sum += 1                                          #每类图片的数目
                all_class_images += 1                                   #所有类图片的数目
             
            # 说明的json文件的class_detail数据
            class_detail_list['class_name'] = dict_data[class_dir]             #类别名称
            class_detail_list['class_label'] = class_label          #类别标签
            class_detail_list['class_eval_images'] = eval_sum       #该类数据的测试集数目
            class_detail_list['class_trainer_images'] = trainer_sum #该类数据的训练集数目
            class_detail.append(class_detail_list)  
            #初始化标签列表
            train_parameters['label_dict'][str(class_label)] = dict_data[class_dir]
            
    #初始化分类数
    train_parameters['class_dim'] = class_dim

    #乱序  
    random.shuffle(eval_list)
    with open(eval_list_path, 'a') as f:
        for eval_image in eval_list:
            f.write(eval_image) 
            
    random.shuffle(trainer_list)
    with open(train_list_path, 'a') as f2:
        for train_image in trainer_list:
            f2.write(train_image) 

    for ii in range(40):
        if ii==39:
            label_list.append(str(ii))
        else:
            label_list.append(str(ii) + "\n")
    with open(label_list_path,'a') as f1:
        for label in label_list:
            f1.write(label)
    
    print ('生成数据列表完成！')

In [7]:
target_path=train_parameters['target_path']
train_list_path=train_parameters['train_list_path']
eval_list_path=train_parameters['eval_list_path']
label_list_path=train_parameters['label_list_path']
get_data_list(target_path,train_list_path,eval_list_path)

生成数据列表完成！


In [8]:
import paddle
import paddlehub as hub
module = hub.Module(name="resnet_v2_50_imagenet")

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlehub/contrib/ppdet/utils/coco_eval.py:26: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  

In [9]:
from paddlehub.dataset.base_cv_dataset import BaseCVDataset
   
class DemoDataset(BaseCVDataset):	
   def __init__(self):	
       self.dataset_dir = "data/rubbish/train"
       super(DemoDataset, self).__init__(
           base_path=self.dataset_dir,
           train_list_file="train.txt",
           validate_list_file="eval.txt",
           test_list_file="eval.txt",
           label_list_file="label.txt",
           )
dataset = DemoDataset()

In [10]:
data_reader = hub.reader.ImageClassificationReader(
    image_width=module.get_expected_image_width(),
    image_height=module.get_expected_image_height(),
    images_mean=module.get_pretrained_images_mean(),
    images_std=module.get_pretrained_images_std(),
    dataset=dataset 
 )

[2021-05-25 22:45:56,988] [    INFO] - Dataset label map = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14, '15': 15, '16': 16, '17': 17, '18': 18, '19': 19, '20': 20, '21': 21, '22': 22, '23': 23, '24': 24, '25': 25, '26': 26, '27': 27, '28': 28, '29': 29, '30': 30, '31': 31, '32': 32, '33': 33, '34': 34, '35': 35, '36': 36, '37': 37, '38': 38, '39': 39}


In [11]:
config = hub.RunConfig(
    use_cuda=True,
    num_epoch =5, 
    checkpoint_dir="cv_finetune_turtorial_demo",
    batch_size=128, 
    eval_interval=20,
    strategy=hub.finetune.strategy.DefaultFinetuneStrategy()    
) 

[2021-05-25 22:45:56,994] [    INFO] - Checkpoint dir: cv_finetune_turtorial_demo


In [12]:
input_dict, output_dict, program = module.context(trainable=True)
img = input_dict["image"]
feature_map = output_dict["feature_map"]
feed_list = [img.name]

task = hub.ImageClassifierTask(
    data_reader=data_reader,
    feed_list=feed_list,
    feature=feature_map,
    num_classes=dataset.num_labels,
    config=config)

[2021-05-25 22:45:57,065] [    INFO] - 267 pretrained paramaters loaded by PaddleHub


In [13]:
run_states = task.finetune_and_eval()

[2021-05-25 22:46:03,244] [    INFO] - Strategy with slanted triangle learning rate, L2 regularization, 
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/executor.py:811: UserWarning: There are no operators in the program to be executed. If you pass Program manually, please use fluid.program_guard to ensure the current Program is being used.
  warnings.warn(error_info)
[2021-05-25 22:46:05,020] [    INFO] - Try loading checkpoint from cv_finetune_turtorial_demo/ckpt.meta
[2021-05-25 22:46:05,021] [    INFO] - PaddleHub model checkpoint not found, start from scratch...
[2021-05-25 22:46:05,072] [    INFO] - PaddleHub finetune start
[2021-05-25 22:46:22,488] [   TRAIN] - step 10 / 559: loss=3.23685 acc=0.23828 [step/sec: 0.64]
[2021-05-25 22:46:38,426] [   TRAIN] - step 20 / 559: loss=2.01761 acc=0.58984 [step/sec: 0.68]
[2021-05-25 22:46:38,427] [    INFO] - Evaluation on dev dataset start
share_vars_from is set, scope is ignored.
[2021-05-25 22:46:40,224]

In [14]:
!unzip -q -o ./data/data35095/test.zip -d ./data/result

In [15]:
data = []
for line in open("data/result/testpath.txt","r"): #设置文件对象并读取每一行文件
     line=line.strip("\n")           #删除首尾的”\n”
     data.append("data/result/test/"+line)               #将每一行文件加入到data中
 
data


['data/result/test/test1.jpg',
 'data/result/test/test2.jpg',
 'data/result/test/test3.jpg',
 'data/result/test/test4.jpg',
 'data/result/test/test5.jpg',
 'data/result/test/test6.jpg',
 'data/result/test/test7.jpg',
 'data/result/test/test8.jpg',
 'data/result/test/test9.jpg',
 'data/result/test/test10.jpg',
 'data/result/test/test11.jpg',
 'data/result/test/test12.jpg',
 'data/result/test/test13.jpg',
 'data/result/test/test14.jpg',
 'data/result/test/test15.jpg',
 'data/result/test/test16.jpg',
 'data/result/test/test17.jpg',
 'data/result/test/test18.jpg',
 'data/result/test/test19.jpg',
 'data/result/test/test20.jpg',
 'data/result/test/test21.jpg',
 'data/result/test/test22.jpg',
 'data/result/test/test23.jpg',
 'data/result/test/test24.jpg',
 'data/result/test/test25.jpg',
 'data/result/test/test26.jpg',
 'data/result/test/test27.jpg',
 'data/result/test/test28.jpg',
 'data/result/test/test29.jpg',
 'data/result/test/test30.jpg',
 'data/result/test/test31.jpg',
 'data/result/tes

In [17]:
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.image as image
#显示matplotlib生成的图形
%matplotlib inline

index = 0
label_map = dataset.label_dict()

run_states = task.predict(data=data)
results = [run_state.run_results for run_state in run_states]
file_handle=open('data/result/model_result.txt',mode='w')
for batch_result in results:
    batch_result = np.argmax(batch_result, axis=2)[0]
    for result in batch_result:
        index += 1
        result = label_map[result]
        print("input %i is %s, and the predict result is %s" %
              (index, data[index - 1], result))
        file_handle.write(result+"\n")



[2021-05-25 23:07:52,100] [    INFO] - PaddleHub predict start
[2021-05-25 23:07:52,101] [    INFO] - The best model has been loaded
[2021-05-25 23:07:58,487] [    INFO] - PaddleHub predict finished.


input 1 is data/result/test/test1.jpg, and the predict result is 29
input 2 is data/result/test/test2.jpg, and the predict result is 1
input 3 is data/result/test/test3.jpg, and the predict result is 4
input 4 is data/result/test/test4.jpg, and the predict result is 23
input 5 is data/result/test/test5.jpg, and the predict result is 5
input 6 is data/result/test/test6.jpg, and the predict result is 0
input 7 is data/result/test/test7.jpg, and the predict result is 1
input 8 is data/result/test/test8.jpg, and the predict result is 31
input 9 is data/result/test/test9.jpg, and the predict result is 39
input 10 is data/result/test/test10.jpg, and the predict result is 34
input 11 is data/result/test/test11.jpg, and the predict result is 31
input 12 is data/result/test/test12.jpg, and the predict result is 17
input 13 is data/result/test/test13.jpg, and the predict result is 37
input 14 is data/result/test/test14.jpg, and the predict result is 22
input 15 is data/result/test/test15.jpg, an